#Final Project-Book recommender

1.First, we are going to explore the datasets
We are going to be working with 3 datasets:
+Goodreads dataset (source: Kaggle)
+Amazon Books dataset 1 (source: Kaggle)
+Amazon Bookss dataset 2 (source: Kaggle)

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from pandas.plotting import table
pd.set_option('display.max_columns', None)

In [ ]:
pip install plotly

In [ ]:
pip install kaleido

In [ ]:
import plotly.figure_factory as ff

##GoodReads Dataset

In [ ]:
database_1=pd.read_csv(r"C:\Users\torra\IH-Labs\Final Project\books.csv",sep=',',index_col=0,on_bad_lines='warn')

In [ ]:
database_GR=pd.DataFrame(database_1)

Note on ISBN numbers:
The ISBN identifies the registrant as well as the specific title, edition and format.Any book made publicly available, whether for sale or on a gratis basis, can be identified by ISBN.ISBNs were 10 digits in length up to the end of December 2006, but since 1 January 2007 they now always consist of 13 digits.
https://www.isbn-international.org/content/what-isbn/10

In [ ]:
database_GR.reset_index()

Checking for null values in Goodreads dataframe

In [ ]:
database_GR.isna().sum()

Goodnews!! This dataframe is clean of null values

###Cleaning proces by column

Title Column

In [ ]:
database_GR["title"].unique()

In [ ]:
database_GR["title"].value_counts().idxmax()

In [ ]:
database_GR["title"].mode()

##20 most reviewed books

In [ ]:
n = 20
database_GR["title"].value_counts()[:n].index.tolist()

In [ ]:
n=20
Most_mentionGR=pd.DataFrame(database_GR["title"].value_counts()[:n].index.tolist())

In [ ]:
Most_mentionGR

In [ ]:
Most_mentionGR.set_axis(['Goodreads'], axis='columns')

###Column Authors

In [ ]:
database_GR["authors"].unique()

In [ ]:
database_GR["authors"].value_counts().idxmax()

In [ ]:
database_GR["title"].mode()

In [ ]:
n = 20
database_GR["authors"].value_counts()[:n].index.tolist()

In [ ]:
database_GR['authors']=database_GR['authors'].str.split('/').str.get(0)



###Column average_rating

In [ ]:
database_GR["average_rating"].unique()

In [ ]:
database_GR.average_rating.hist(bins=10)
plt.title('Histogram of average rating (Goodreads)')
plt.xlabel('Average Rating')
plt.ylabel('Frequency')

The variable average rating present a distribution skewed to the right.


###Column language_code

In [ ]:
database_GR["language_code"].unique()

In [ ]:
database_GR["language_code"].value_counts()

The language code has a lot of subcategories, it would be useful to trim it down.
Will leave english, french, german and spanish and others.


In [ ]:
database_GR['language_code']=database_GR['language_code'].replace(['en-US','en-GB','en-CA'], 'eng')

In [ ]:
database_GR['language_code']=database_GR['language_code'].replace(['mul', 'grc', 'enm','jpn', 'ara', 'nl', 'zho', 'lat', 'por', 'srp', 'ita', 'rus', 'msa', 'glg',
       'wel', 'swe', 'nor', 'tur', 'gla', 'ale'], 'other')

In [ ]:
database_GR["language_code"].unique()

###Column num_pages

###Column ratings_counts

In [ ]:
database_GR["ratings_count"].median()

In [ ]:
database_GR["ratings_count"]

In [ ]:
database_GR.ratings_count.hist()
plt.title('Histogram of ratings count')
plt.xlabel('Ratings Count')
plt.ylabel('Frequency')

We are going to drop this column, as we have the average rating that is a more indicative.


###Column text_reviews_count

In [ ]:
database_GR["text_reviews_count"].median()

In [ ]:
database_GR["text_reviews_count"].describe()

In [ ]:
database_GR.text_reviews_count.hist()
plt.title('Histogram of text reviews count')
plt.xlabel('Text Review Count')
plt.ylabel('Frequency')

We change the column "text reviews count", if the amount of reviews is larger than the median we input 1 if not 0.
We take the amount of reviews a books has as a level of engagement.
1:high engagement
0:low engagement

In [ ]:
database_GR["text_reviews_count"] = np.where(database_GR["text_reviews_count"]>database_GR["text_reviews_count"].median(), 1,0)

###Column publication_date

We are going to keep only the year of publication

In [ ]:

database_GR['publication_date']=database_GR['publication_date'].str.split('/').str.get(2)
#df[0].str.split('--').str.get(0).str[-5:].str.strip()

In [ ]:
database_GR['publication_date']

##EDA Analysis

###Number of published books by author (in the database)

####Top 10

In [ ]:
database_GR.groupby('authors')['title'].count().sort_values(ascending=False).head(10)

In [ ]:
authors_10=[database_GR.groupby('authors')['title'].count().sort_values(ascending=False).head(10)]


In [ ]:
authors_10

In [ ]:
authors_pd=pd.DataFrame(authors_10, columns=['authors'])

###Average rating per author

In [ ]:
authors_rating=[database_GR.groupby('authors')['average_rating'].mean().sort_values(ascending=False).head(10)]

In [ ]:
authors_rating

We can observe that the authors with the highest score are not the same that are listed as the most titles published.

###Authors with the most amount of written reviews

In [ ]:
database_GR.groupby('authors')['text_reviews_count'].count().sort_values(ascending=False).head(10)

In [ ]:
top_authors_gr=database_GR.groupby('authors')['text_reviews_count'].count().sort_values(ascending=False).head(10)

In [ ]:
top_authors_gr.plot.bar()

Books per publisher

In [ ]:
database_GR.groupby('publisher')['title'].count().sort_values(ascending=False).head(10)

In [ ]:
publishers_10_GR=database_GR.groupby('publisher')['title'].count().sort_values(ascending=False).head(10)

In [ ]:
publishers_10_GR.plot.bar()

##Model

In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate



df = database_GR

reader = Reader(rating_scale=(1, 5))  # Rating scale is 1 to 5
svddata = Dataset.load_from_df(database_GR[['title', 'isbn', 'average_rating']], reader)

svd = SVD()

trainset = svddata.build_full_trainset()
svd.fit(trainset)


isbn = '8497646983'
title = 'Las aventuras de Tom Sawyer'

pred = svd.predict( title, isbn)


print(f'Predicted rating for {title} by {isbn}: {pred.est}')

Amazon dataset 1
Cleaning process

In [ ]:
database_2=pd.read_csv(r"C:\Users\torra\IH-Labs\Final Project\books_data.csv",sep=',',index_col=0,on_bad_lines='warn')

In [ ]:
database_AMZ1=pd.DataFrame(database_2)

In [ ]:
database_AMZ1

Transform the style of the dataframe

In [ ]:
database_AMZ1 = database_AMZ1.reset_index()
database_AMZ1




Checking for null values in Amazon 1  dataframe

In [ ]:
database_AMZ1.info()

In [ ]:
database_AMZ1=database_AMZ1.drop(['previewLink'], axis=1)

In [ ]:
database_AMZ1=database_AMZ1.drop(['infoLink'], axis=1)

In [ ]:
database_AMZ1.isna().sum()

In [ ]:
#Strip columns with parenthesis

In [ ]:
database_AMZ1['authors'] = database_AMZ1['authors'].str.strip('[]')

In [ ]:
database_AMZ1['authors'] = database_AMZ1['authors'].str.strip("''")

In [ ]:
database_AMZ1['categories'] = database_AMZ1['categories'].str.strip('')

In [ ]:
database_AMZ1['categories'] = database_AMZ1['categories'].str.strip("''")

In [ ]:
database_AMZ1.head(5)

This dataset contains a considerable amount of null values

In [ ]:
#We want to know how many null values are compare to the total inputs in the column
database_AMZ1.isna().sum()/len(database_AMZ1)

We are going to fill the null values in the column "description" with the phrase "no description"

In [ ]:
database_AMZ1['description']=database_AMZ1['description'].fillna('no_description')

We are going to fill the null values in the column "authors" with the word "unknown"

In [ ]:
database_AMZ1['authors']=database_AMZ1['authors'].fillna('unknown')

We are going to fill the null values in the column "image" with the word "without"

In [ ]:
database_AMZ1['image']=database_AMZ1['image'].fillna('without')

We are going to fill the null values in the column "publisher" with the word "unknown"

In [ ]:
database_AMZ1['publisher']=database_AMZ1['publisher'].fillna('unknown')

We are going to fill the null values in the column "publishedDate" with the word "unknown"

In [ ]:
database_AMZ1['publishedDate']=database_AMZ1['publishedDate'].fillna('unknown')

We are going to fill the null values in the column "categories" with the phrase "no_category"

In [ ]:
database_AMZ1['categories']=database_AMZ1['categories'].fillna('no_category')

##Column analysis

###Column Title

In [ ]:
database_AMZ1["Title"].unique()

In [ ]:
database_AMZ1['Title'].value_counts()

In [ ]:
database_AMZ1["Title"].isna().sum()

###Column ratingsCount

This column is similar to the column from the Goodreads column text_reviews_count.We are going to apply the same type of transformation.

In [ ]:
database_AMZ1["ratingsCount"] = np.where(database_AMZ1["ratingsCount"]>database_AMZ1["ratingsCount"].median(), 1,0)

In [ ]:
database_AMZ1

##EDA Analysis

###Top 10 authors by titles

In [ ]:
database_AMZ1.groupby('authors')['Title'].count().sort_values(ascending=False).head(10)

In [ ]:
top_authors_amz1=database_AMZ1.groupby('authors')['Title'].count().sort_values(ascending=False).head(10)

In [ ]:
top_authors_amz1.info()

In [ ]:
top_authors_pdamz1=top_authors_amz1.to_frame()

In [ ]:
top_authors_pdamz1['authors'] = top_authors_pdamz1['authors'].str.strip('['']')

###Top 10 publishers by titles

In [ ]:
database_AMZ1.groupby('publisher')['Title'].count().sort_values(ascending=False).head(10)

In [ ]:
top_publishers_amz=database_AMZ1.groupby('publisher')['Title'].count().sort_values(ascending=False).head(10)

In [ ]:
top_publishers_amz.plot.bar()

##Amazon dataset 2

In [ ]:
database_3=pd.read_csv(r"C:\Users\torra\IH-Labs\Final Project\Books_rating.csv",sep=',',index_col=0,on_bad_lines='warn')

In [ ]:
database_AMZ2=pd.DataFrame(database_3)

In [ ]:
database_AMZ2.reset_index()

Drop Columns

In [ ]:
database_AMZ2=database_AMZ2.drop(['Price'], axis=1)

In [ ]:
database_AMZ2=database_AMZ2.drop(['review/time'], axis=1)

In [ ]:
database_AMZ2=database_AMZ2.drop(['review/summary'], axis=1)

In [ ]:
database_AMZ2=database_AMZ2.drop(['profileName'], axis=1)

In [ ]:
database_AMZ2=database_AMZ2.drop(['review/helpfulness'], axis=1)

Checking for null values in Amazon 2  dataframe

In [ ]:
database_AMZ2.isna().sum()

Treating null values 

###Column title

In [ ]:
database_AMZ2['Title']=database_AMZ2['Title'].fillna('unknown')

###Column User_id

In [ ]:
database_AMZ2['User_id']=database_AMZ2['User_id'].fillna('unknown')

###Column review/text 

In [ ]:
database_AMZ2['review/text']=database_AMZ2['review/text'].fillna('no_review')

In [ ]:
database_AMZ2=database_AMZ2.reset_index()

In [ ]:
database_AMZ2

###Concatenation of Amazon's databases

In [ ]:
database_AMZ_complete=pd.merge(database_AMZ1,database_AMZ2, how="inner",on=['Title'])

In [ ]:
database_AMZ_complete=database_AMZ_complete.rename(columns={'review/score':'score'})


In [ ]:
database_AMZ_complete.info()

In [ ]:
top_authors_AMz=database_AMZ_complete.groupby('authors')['review/text'].count().sort_values(ascending=False).head(10)

In [ ]:
top_authors_AM=top_authors_AMz.to_frame

In [ ]:
database_AMZ_complete['score']=database_AMZ_complete['score'].astype(int)

After the concatenation of Amazon's dataset, we are doing a similarity check with the Goodreads database using 

First, we are creating a database only with author and title from goodreads and amazon, we are comparing for similarity.

In [ ]:
GR_comp=database_GR[['title','authors']]

In [ ]:
GR_comp=GR_comp.rename(columns={'title': 'title_gr', 'authors': 'authors_gr'})

In [ ]:
AMZ_comp=database_AMZ_complete[['Title','authors']]

In [ ]:
comp_tot=pd.concat([GR_comp, AMZ_comp], axis=1)

In [ ]:
comp_tot.head(5)

In [ ]:
database_AMZ_complete.head(10)

We save the complete database ( both dataframes containing amazon's info)

In [ ]:
database_AMZ_complete.to_csv(r"C:\Users\torra\IH-Labs\Final Project\AMZ_complete.csv", sep=",",encoding="utf-8")

KNN method- using surprise library

In [ ]:
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [ ]:

!pip install surprise

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import networkx as nx
from collections import defaultdict
from surprise import Dataset, Reader, SVD, accuracy, KNNBaseline, get_dataset_dir
from surprise.model_selection import train_test_split

In [ ]:
Amazon_books=pd.read_csv(r"C:\Users\torra\IH-Labs\Final Project\AMZ_complete.csv", sep=",",encoding="utf-8")

In [ ]:
n=20
Most_mentionAM=pd.DataFrame(Amazon_books["Title"].value_counts()[:n].index.tolist())

In [ ]:
Most_mentionAM

In [ ]:
Most_mentionAM.set_axis(['Amazon'], axis='columns')

In [ ]:
database_20=pd.concat([Most_mentionAM,Most_mentionGR],axis='columns')

###Comparison most bought/review books in each database
(review in goodreads/bought in amazon)

In [ ]:
database_20

In [ ]:
database_20=database_20.set_axis(['Amazon','Goodreads'], axis='columns')

In [ ]:
#export table as png
ax = plt.subplot(111, frame_on=False) 
ax.xaxis.set_visible(False) 
ax.yaxis.set_visible(False) 
table(ax, database_20, loc="center")  
plt.savefig("Top_20_books .png")
#Let's enhace the size of the table
fig = ff.create_table(database_20)
fig.update_layout(
    autosize=False,
    width=1000,
    height=800)

fig.write_image("Top_20_books.png", scale=4)

AS we can see in both cases there is a mix of classics and genre betsellers, a lot of them appear in both, just in a different order.
This indicates that the users of goodreads and the buyers of amazon have a similar sensibility, in many cases they might use both platforms.

In [ ]:
Amazon_sample = Amazon_books.sample(frac=0.4)

In [ ]:
Amazon_sample.to_csv(r"C:\Users\torra\IH-Labs\Final Project\AMZ_sample.csv", sep=",",encoding="utf-8")

##Surprise Model/Method

In [ ]:
#Define the reader for the program
reader = Reader(rating_scale=(1, 5))

In [ ]:
data = Dataset.load_from_df(Amazon_sample[['User_id', 'Id', 'review/score']], reader)

In [ ]:
#We split the data
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
# Instantiate the algorithm
algo = SVD()

In [ ]:
# Train the model
algo.fit(trainset)

In [ ]:

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
predictions = algo.test(testset)

In [ ]:
top_n = get_top_n(predictions, n=5)
top_n

In [ ]:
rmse = accuracy.rmse(predictions)

In [ ]:
#We ask for two recommendations
predictions[:2]

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import re

In [ ]:
text = Amazon_sample['review/text'].description[3]

In [ ]:
processed_text = re.sub(r'[^\w\s]', '', text.lower())
processed_text = re.sub(r'[^\w\s]', '', processed_text)
processed_text = re.sub('\s+', ' ', processed_text)

In [ ]:
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color='white').generate(processed_text)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
Amazon_sample[Amazon_sample['Id'] == '0802139744']

In [ ]:
id_to_title = dict(zip(Amazon_sample['Id'], Amazon_sample['Title']))

In [ ]:
id_to_title

In [ ]:
id_to_title_pd=pd.DataFrame.from_dict(id_to_title)

In [ ]:
Amazon_books.info()

Changes to be made to the matrix

In [ ]:
Amazon_books['ratingsCount']=Amazon_books['ratingsCount'].astype(str)

In [ ]:
Amazon_books['review/score']=Amazon_books['review/score'].astype(str)

In [ ]:
columns_for_analysis=['Title','ratingsCount','publisher','review/score']

In [ ]:
Amazon_sample = Amazon_books.sample(frac=0.4)

In [ ]:
Amazon_sample.info()

In [ ]:
def new_features(data):
    features=[]
    for i in range(0,data.shape[0]):
        features.append(data['Title'][i]+" "+data['publisher'][i]+" "+data['ratingsCount'][i]+" "+data['review/score'])
       
    return features

In [ ]:
Amazon_sample['new_features']=new_features(Amazon_sample)

##NEW YORk TIMES BEST SELLERS LIST API

In [ ]:
from pandas import json_normalize
import requests
import getpass




In [ ]:
pip install --upgrade pynytimes


In [ ]:
from pynytimes import NYTAPI

##NYTAPI development citation
@software{Den_Heijer_pynytimes_2023,
    author = {Den Heijer, Micha},
    license = {MIT},
    title = {{pynytimes}},
    url = {https://github.com/michadenheijer/pynytimes},
    version = {0.10.0},
    year = {2023},
    doi = {10.5281/zenodo.7821090}

In [ ]:
apikey = getpass.getpass()
nyt = NYTAPI( apikey, parse_dates=True)

In [ ]:
pip install DateTimeRange

In [ ]:
import datetime
from datetimerange import DateTimeRange

In [ ]:
books = nyt.best_sellers_list(name="combined-print-and-e-book-fiction", date=datetime.datetime(2019,1,1))

In [ ]:
date=datetime.datetime(2019,1,1)

In [ ]:
date

In [ ]:
books

In [ ]:
from datetime import date, timedelta
from random import randint
from time import sleep

date_list=pd.date_range(start="2014-01-01", freq='ME',periods=10).to_list()

def get_list(dates):

    ##dates=date_list

    List=[]
    
    for i in dates:
        books1 = nyt.best_sellers_list(name="combined-print-and-e-book-fiction", date=i)
    
        list.append(books1)
        sleep(randint(1,3000)/1000) 
        print(i)
    return List




 



In [ ]:
date_list1=[(2014,1), (2014,2),(2014,3), (2014,4),(2014,5), (2014,6),(2014,7), (2014,8),(2014,9), (2014,10),(2014,11), (2014,12),(2015,1), (2015,2),(2015,3), (2015,4),(2015,5), (2015,6),(2015,7), (2015,8),(2015,9), (2015,10),(2015,11), (2015,12),(2016,1), (2016,2),(2016,3), (2016,4),(2016,5), (2016,6),(2016,7), (2016,8),(2016,9), (2016,10),(2016,11), (2016,12),(2017,1), (2017,2),(2017,3), (2017,4),(2017,5), (2017,6),(2017,7), (2017,8),(2017,9), (2017,10),(2017,11), (2017,12),(2018,1), (2018,2),(2018,3), (2018,4),(2018,5), (2018,6),(2018,7), (2018,8),(2018,9), (2018,10),(2018,11), (2018,12),(2019,1), (2019,2),(2019,3), (2019,4),(2019,5), (2019,6),(2019,7), (2019,8),(2019,9), (2019,10),(2019,11), (2019,12),(2020,1), (2020,2),(2020,3), (2020,4),(2020,5), (2020,6),(2020,7), (2020,8),(2020,9), (2020,10),(2020,11), (2020,12)]
List=[]
for year, month in date_list1:
   books = nyt.best_sellers_list(name="combined-print-and-e-book-fiction", date=datetime.datetime(year,month,1))
   List.append(books)
   sleep(randint(1,3000)/1000) 
   print(year,month)
   

In [ ]:
List

In [ ]:
NYT_df= pd.json_normalize(List[1])

In [ ]:
flatten_NYT = pd.DataFrame()

for el in List:
    NYT_df= pd.json_normalize(el) 
    pd.concat([flatten_NYT,NYT_df], axis=0, ignore_index=True)

In [ ]:
flatten_NYT.info()

In [ ]:
NYT_df.info()

In [ ]:
NYT_df

In [ ]:
print(List[2])

In [ ]:
NYT_df

In [ ]:
List[1]

In [ ]:
print(NYT_df)

In [ ]:
NYT_df.to_csv(NYT_df,sep=",",index=False, encoding="utf-8")

In [ ]:
NYT_df.to_pickle(path="C:\Users\torra\IH-Labs\Final_Project", compression="infer", protocol=5, storage_options=None)

In [ ]:
List_pd= pd.json_normalize(List)

In [ ]:
#https://api.nytimes.com/svc/books/v3/lists/full-overview.json